In [1]:
import os
from pathlib import Path

import polars as pl
from helpers import db_to_polars, polars_to_db
from polars import col as c
from rapidfuzz import process

pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(50)
rot = Path(os.environ.get("ROTSTI"))
datasti = rot / "database"
prosjektsti = rot / "dekning-2024"

In [2]:
tilbyderliste = []  # Hvilke tilbydere skal vi koble til - skriv tilbyderliste = 'alle' for alle tilbyderne

In [3]:
status = db_to_polars("dekning24", "status")
tilbid_mapping = {}
for rad in status.to_dicts():
    tilbid_mapping[rad["tilb"]] = rad["tilbid"]

In [4]:
abdata = {1: set(), 2: set()}

In [5]:
def fiberfiks(tilb, abfiberdata):
    liste = []
    for rad in abfiberdata.to_dicts():
        liste.append(
            {
                "radnr": rad["radnr"],
                "adrid": rad["adrid"],
                "hc": True,
                "eier": rad["eier"],
                "nodeid": "",
                "komnr": rad["komnr"],
                "gatenavn": rad["gatenavn"],
                "husnr": rad["husnr"],
                "bokstav": rad["bokstav"],
                "postnr": rad["postnr"],
                "poststed": rad["poststed"],
                "gnr": rad["gnr"],
                "bnr": rad["bnr"],
                "fnr": rad["fnr"],
                "unr": rad["unr"],
                "koblet": False,
                "dup": None,
            }
        )
    fiberdata = pl.DataFrame(liste).select(
        c.radnr,
        c.adrid.cast(pl.Int64),
        c.hc,
        c.eier,
        c.nodeid,
        c.komnr.cast(pl.Int16),
        c.gatenavn,
        c.husnr.cast(pl.Int16),
        c.bokstav,
        c.postnr.cast(pl.Int16),
        c.poststed,
        c.gnr.cast(pl.Int16),
        c.bnr.cast(pl.Int16),
        c.fnr.cast(pl.Int16),
        c.unr.cast(pl.Int16),
        c.koblet,
        c.dup,
    )
    fiberdata.write_parquet(tilb / "1formatert" / "fiberdata.parquet")
    fiberdata.write_excel(tilb / "1formatert" / "fiberdata.xlsx")

In [6]:
def kabelfiks(tilb, abkabeldata):
    liste = []
    for rad in abkabeldata.to_dicts():
        liste.append(
            {
                "radnr": rad["radnr"],
                "adrid": rad["adrid"],
                "eier": rad["eier"],
                "komnr": rad["komnr"],
                "gatenavn": rad["gatenavn"],
                "husnr": rad["husnr"],
                "bokstav": rad["bokstav"],
                "postnr": rad["postnr"],
                "poststed": rad["poststed"],
                "gnr": rad["gnr"],
                "bnr": rad["bnr"],
                "fnr": rad["fnr"],
                "unr": rad["unr"],
                "koblet": False,
                "dup": None,
            }
        )
    kabeldata = pl.DataFrame(liste).select(
        c.radnr,
        c.adrid.cast(pl.Int64),
        c.eier,
        c.komnr.cast(pl.Int16),
        c.gatenavn,
        c.husnr.cast(pl.Int16),
        c.bokstav,
        c.postnr.cast(pl.Int16),
        c.poststed,
        c.gnr.cast(pl.Int16),
        c.bnr.cast(pl.Int16),
        c.fnr.cast(pl.Int16),
        c.unr.cast(pl.Int16),
        c.koblet,
        c.dup,
    )
    kabeldata.write_parquet(tilb / "1formatert" / "kabeldata.parquet")
    kabeldata.write_excel(tilb / "1formatert" / "kabeldata.xlsx")

In [7]:
def oppdater_status(tilb, filnavn, data, xxx, ant_adrid, ekom, ant_like_adrid):
    if filnavn == "fiberdata":
        fiberdata = db_to_polars("dekning24", "fiber").filter(c.tilb != tilb)
        data2 = data.filter(c.adrid.gt(0) & ~c.adrid.is_in(abdata[1]) & ~c.remove)
        data = {
            "tilbid": tilbid_mapping[tilb],
            "tilb": tilb,
            "rader": len(data),
            "xxx": xxx + ant_like_adrid,
            "koblet": len(data.filter(c.adrid.gt(0) & ~c.remove)),
            "ukoblet": len(data.filter(c.adrid == 0)) - xxx,
            "eier": len(data.filter(c.adrid.gt(0) & c.eier.eq("") & ~c.remove)),
            "ab": len(abdata[1]),
            "hc": len(data2.filter(c.hc)),
            "hp": len(data2.filter(~c.hc)),
            "est": 0,
            "bokst": 0,
            "abklynge": 0,
            "hull": 0,
            "nye": 0,
            "mistet": 0,
            "hustot": 0,
            "husny": 0,
            "huslost": 0,
            "adrid": ant_adrid,
        }
        fiberdata = pl.concat([fiberdata, pl.DataFrame([data])])
        polars_to_db(fiberdata, "dekning24", "fiber")
    if filnavn == "kabeldata":
        kabeldata = db_to_polars("dekning24", "kabel").filter(c.tilb != tilb)
        data = {
            "tilbid": tilbid_mapping[tilb],
            "tilb": tilb,
            "rader": len(data),
            "xxx": xxx + ant_like_adrid,
            "koblet": len(data.filter(c.adrid.gt(0) & ~c.remove)),
            "ukoblet": len(data.filter(c.adrid == 0)) - xxx,
            "eier": len(data.filter(c.adrid.gt(0) & c.eier.eq("") & ~c.remove)),
            "ab": len(abdata[2]),
            "ekstra": len(data.filter(c.adrid.gt(0) & ~c.adrid.is_in(abdata[2]) & ~c.remove)),
            "abklynge": 0,
            "nye": 0,
            "mistet": 0,
            "hustot": 0,
            "husny": 0,
            "huslost": 0,
            "adrid": ant_adrid,
        }
        kabeldata = pl.concat([kabeldata, pl.DataFrame([data])])
        polars_to_db(kabeldata, "dekning24", "kabel")
    if filnavn == "abdata":
        abdata[1] |= set(data.filter(c.adrid > 0).filter(c.tek == 1)["adrid"])
        abdata[2] |= set(data.filter(c.adrid > 0).filter(c.tek == 2)["adrid"])
        abodata = db_to_polars("dekning24", "ab").filter(c.tilb != tilb)
        data = {
            "tilbid": tilbid_mapping[tilb],
            "tilb": tilb,
            "ekom": ekom["tot"].sum(),
            "rader": len(data),
            "koblet": len(data.filter(c.adrid.gt(0))),
            "eier": len(data.filter(c.adrid.gt(0) & c.eier.eq(""))),
            "diff1p": ekom["p1"].sum() - len(data.filter(c.tek == 1).filter(c.ms == "privat")),
            "diff1b": ekom["b1"].sum() - len(data.filter(c.tek == 1).filter(c.ms == "bedrift")),
            "diff2p": ekom["p2"].sum() - len(data.filter(c.tek == 2).filter(c.ms == "privat")),
            "diff2b": ekom["b2"].sum() - len(data.filter(c.tek == 2).filter(c.ms == "bedrift")),
            "diff3p": ekom["p3"].sum() - len(data.filter(c.tek == 3).filter(c.ms == "privat")),
            "diff3b": ekom["b3"].sum() - len(data.filter(c.tek == 3).filter(c.ms == "bedrift")),
            "diff9p": ekom["p9"].sum() - len(data.filter(c.tek > 3).filter(c.ms == "privat")),
            "diff9b": ekom["b9"].sum() - len(data.filter(c.tek > 3).filter(c.ms == "bedrift")),
            "adrid": ant_adrid,
        }
        abodata = pl.concat([abodata, pl.DataFrame([data])])
        polars_to_db(abodata, "dekning24", "ab")

In [8]:
def fiks_dekning(df):
    if "hc" in df.columns:
        data = (
            df.sort(c.adrid, c.hc, c.nodeid, c.eier.str.to_lowercase(), descending=True)
            .with_columns(c.adrid.shift(1).fill_null(0).eq(c.adrid).alias("remove"))
            .with_columns(pl.when(c.adrid == 0).then(pl.lit(False)).otherwise(c.remove).alias("remove"))
        )
    else:
        data = (
            df.sort(c.adrid, c.eier.str.to_lowercase(), descending=True)
            .with_columns(c.adrid.shift(1).fill_null(0).eq(c.adrid).alias("remove"))
            .with_columns(pl.when(c.adrid == 0).then(pl.lit(False)).otherwise(c.remove).alias("remove"))
        )
    return data


In [9]:
def finn_adrid(df, onliste, kobling):
    if onliste == "adrid":
        test = df.filter(c.adrid > 0).join(adr, on="adrid").select(c.radnr, c.adrid)
    else:
        data = df.drop("adrid")
        if "gatenavn" in onliste:
            data = data.filter(c.gatenavn.ne(""))
        test = data.join(adr, on=onliste).select(c.radnr, c.adrid).unique(subset=["radnr"])

    for rad in test.to_dicts():
        kobling[rad["radnr"]] = rad["adrid"]

    radnrliste = list(radnr for radnr in kobling if kobling[radnr] == 0)
    if onliste == "adrid":
        return kobling, df.filter(c.radnr.is_in(radnrliste))

    test = (
        data.filter(c.radnr.is_in(radnrliste)).join(hist, on=onliste).select(c.radnr, c.adrid).unique(subset=["radnr"])
    )
    for rad in test.to_dicts():
        kobling[rad["radnr"]] = rad["adrid"]

    radnrliste = list(radnr for radnr in kobling if kobling[radnr] == 0)
    return kobling, df.filter(c.radnr.is_in(radnrliste))

In [10]:
def kobl_bokstavfeil(df: pl.DataFrame, kobling: dict):
    data = df.drop("adrid").filter(c.gatenavn.ne(""))
    test = (
        data.join(adr, on=["komnr", "gatenavn", "husnr"])
        .sort(c.radnr, c.bokstav_right)
        .unique(subset=["radnr"])
        .select(c.radnr, c.adrid)
    )
    for rad in test.to_dicts():
        kobling[rad["radnr"]] = rad["adrid"]

    radnrliste = list(radnr for radnr in kobling if kobling[radnr] == 0)
    test = (
        data.filter(c.radnr.is_in(radnrliste))
        .join(hist, on=["komnr", "gatenavn", "husnr"])
        .sort(c.radnr, c.bokstav_right)
        .unique(subset=["radnr"])
        .select(c.radnr, c.adrid)
    )
    for rad in test.to_dicts():
        kobling[rad["radnr"]] = rad["adrid"]

    radnrliste = list(radnr for radnr in kobling if kobling[radnr] == 0)
    return kobling, df.filter(c.radnr.is_in(radnrliste))

In [11]:
def finn_gate(struct: pl.Struct) -> str:
    komnr = struct["komnr"]
    gatenavn = struct["gatenavn"]

    if komnr and komnr in komgater:
        best_match_tuple = process.extractOne(gatenavn, komgater[komnr], score_cutoff=75)
        if best_match_tuple is not None:
            return best_match_tuple[0]

    return ""

In [12]:
adr = pl.read_parquet(datasti / "2024" / "adr.parquet").select(
    c.adrid, c.komnr, c.gatenavn, c.husnr, c.bokstav, c.gnr, c.bnr, c.fnr, c.unr, c.postnr, c.poststed
)
hist = pl.read_parquet(prosjektsti / "prep" / "adressedata" / "adr-hist.parquet")

In [13]:
tot = (
    pl.concat([adr, hist])
    .filter(c.gatenavn.ne(""))
    .select(c.komnr, c.gatenavn)
    .unique()
    .sort(c.komnr, c.gatenavn)
    .group_by(c.komnr)
    .agg(c.gatenavn)
)
komgater = {}
for kom in tot.to_dicts():
    komgater[kom["komnr"]] = kom["gatenavn"]

In [ ]:
for tilb in (prosjektsti / "innlesing").iterdir():
    if tilbyderliste != "alle" and tilb.name not in tilbyderliste:
        continue
    formaterte_filer = set([fil.stem for fil in (tilb / "1formatert").iterdir()])
    koblede_filer = set([fil.stem for fil in (tilb / "2koblet").iterdir()])
    if koblede_filer:
        continue
    filer = list(formaterte_filer - {"fibernoder"})
    if "abdata" in filer and ("fiberdata" not in filer or "kabeldata" not in filer):
        abdf = pl.read_parquet(tilb / "1formatert" / "abdata.parquet")
        abdf_fiber = abdf.filter(c.tek == 1)
        if len(abdf_fiber) > 0 and "fiberdata" not in filer:
            fiberfiks(tilb, abdf_fiber)
            filer.append("fiberdata")
        abdf_kabel = abdf.filter(c.tek == 2)
        if len(abdf_kabel) > 0 and "kabeldata" not in filer:
            kabelfiks(tilb, abdf_kabel)
            filer.append("kabeldata")
    for fil in sorted(filer):
        df = pl.read_parquet(tilb / "1formatert" / f"{fil}.parquet")
        ukoblet = df.filter(c.adrid.gt(0) | c.komnr.gt(0) | c.postnr.gt(0)).filter(c.dup.is_null()).sort(c.radnr)
        ant_xxx = len(df) - len(ukoblet)
        print(f"{tilb.name} - {fil} har {ant_xxx} duplikater eller ugyldige adresser")
        kobling = {x: 0 for x in df["radnr"]}

        kobling, ukoblet = finn_adrid(ukoblet, "adrid", kobling)
        ant_adrid = len(set([x for x in kobling.values() if x > 0]))
        kobling, ukoblet = finn_adrid(ukoblet, ["komnr", "gatenavn", "husnr", "bokstav"], kobling)
        kobling, ukoblet = finn_adrid(ukoblet, ["poststed", "gatenavn", "husnr", "bokstav"], kobling)
        kobling, ukoblet = finn_adrid(ukoblet, ["postnr", "gatenavn", "husnr", "bokstav"], kobling)
        kobling, ukoblet = finn_adrid(ukoblet, ["komnr", "gnr", "bnr", "fnr", "unr"], kobling)
        kobling, ukoblet = finn_adrid(ukoblet, ["poststed", "gnr", "bnr", "fnr", "unr"], kobling)
        kobling, ukoblet = finn_adrid(ukoblet, ["postnr", "gnr", "bnr", "fnr", "unr"], kobling)
        kobling, ukoblet = kobl_bokstavfeil(ukoblet, kobling)

        sitron = (
            ukoblet.with_columns(
                pl.struct(["komnr", "gatenavn"]).map_elements(finn_gate, return_dtype=pl.Utf8).alias("korrgate")
            )
            .rename({"gatenavn": "oldgatenavn"})
            .rename({"korrgate": "gatenavn"})
        )

        kobling, sitron = finn_adrid(sitron, ["komnr", "gatenavn", "husnr", "bokstav"], kobling)
        kobling, sitron = finn_adrid(sitron, ["poststed", "gatenavn", "husnr", "bokstav"], kobling)
        kobling, sitron = finn_adrid(sitron, ["postnr", "gatenavn", "husnr", "bokstav"], kobling)
        data = df.with_columns(c.radnr.replace_strict(kobling).alias("adrid")).with_columns(
            (c.adrid > 0).alias("koblet")
        )
        data_kobling = df.with_columns(koblet=data["koblet"])
        data_kobling.write_excel(tilb / "1formatert" / f"{fil}.xlsx")
        data_kobling.write_parquet(tilb / "1formatert" / f"{fil}.parquet")

        # Hent ekomdata
        ekom_privat = (
            pl.read_excel(
                prosjektsti / "innlesing" / tilb.name / "0raw" / "data.xlsx",
                sheet_name="Abonnement - privat",
                drop_empty_rows=False,
                read_options={"header_row": 2, "n_rows": 9},
                columns=["Helår 2024"],
            )
            .transpose(column_names=["d1", "p2", "x1", "p1", "x2", "d2", "d3", "p3", "d4"])
            .with_columns(p9=c.d1.fill_null(0) + c.d2.fill_null(0) + c.d3.fill_null(0) + c.d4.fill_null(0))
            .select(c.p1.fill_null(0), c.p2.fill_null(0), c.p3.fill_null(0), c.p9.fill_null(0))
        )
        ekom_bedrift = (
            pl.read_excel(
                prosjektsti / "innlesing" / tilb.name / "0raw" / "data.xlsx",
                sheet_name="Abonnement - bedrift",
                drop_empty_rows=False,
                read_options={"header_row": 2, "n_rows": 9},
                columns=["Helår 2024"],
            )
            .transpose(column_names=["d1", "b2", "x1", "b1", "x2", "d2", "d3", "b3", "d4"])
            .with_columns(b9=c.d1.fill_null(0) + c.d2.fill_null(0) + c.d3.fill_null(0) + c.d4.fill_null(0))
            .select(c.b1.fill_null(0), c.b2.fill_null(0), c.b3.fill_null(0), c.b9.fill_null(0))
        )
        ekom = pl.concat([ekom_privat, ekom_bedrift], how="horizontal").with_columns(
            tot=pl.sum_horizontal([c.p1, c.p2, c.p3, c.p9, c.b1, c.b2, c.b3, c.b9])
        )
        ant_like_adrid = 0
        if fil in ["fiberdata", "kabeldata"]:
            data = fiks_dekning(data)  # Fikser inkonsistente data
            ant_like_adrid = len(data.filter(c.remove))

        oppdater_status(tilb.name, fil, data, ant_xxx, ant_adrid, ekom, ant_like_adrid)

        if fil in ["fiberdata", "kabeldata"]:
            data = data.filter(~c.remove).drop("remove")

        print(
            f"{tilb.name} - {fil} - Koblet: {len(data.filter(c.adrid > 0))} Gjenstår: {len(data.filter(c.adrid == 0)) - ant_xxx}"
        )

        removeliste = [
            "radnr",
            "komnr",
            "gatenavn",
            "husnr",
            "bokstav",
            "postnr",
            "poststed",
            "gnr",
            "bnr",
            "fnr",
            "unr",
            "koblet",
            "dup",
        ]

        data.filter(c.adrid > 0).select(c("*").exclude(removeliste)).sort(c.adrid).write_parquet(
            tilb / "2koblet" / f"{fil}.parquet"
        )